In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [8]:
file = pd.read_csv('./data/encoded_data_v2.csv')
file.tail(1)

C:\Users\kdp\AppData\Local\Temp\ipykernel_11244\3315460536.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./data/encoded_data_v2.csv')


,Unnamed: 0,image_name,comment_number,comment,0,1,2,3,4,5,...,68,69,70,71,72,73,74,75,76,77
158914,158914,998845445.jpg,4,A man on a moored blue and white boat with hil...,0,2,0,0,6294,11,...,1,1,1,1,1,1,1,1,1,1


In [9]:
data = []
for k in range(file.shape[0]):
    sample = []
    img_dir = './data/flickr30k_images/'
    img_file = img_dir + file.iloc[k][1] 
    sample.append(img_file)

    capt = []
    for idx in range(78):
        capt.append(file.iloc[k][idx+4])
        # print(file.iloc[k][idx+4])

    sample.append(capt)
    data.append(sample)

In [4]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(ImageCaptioningModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.cnn = models.resnet50(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, images, captions):
        features = self.cnn(images)
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        out, _ = self.rnn(embeddings)
        out = self.fc(out)
        return out


In [5]:
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [6]:
# 이미지와 캡션 로드
image = Image.open(file.iloc[0][1])
image = transform(image).unsqueeze(0)
caption = torch.LongTensor(file.iloc[0][1])  # 예시 캡션

FileNotFoundError: [Errno 2] No such file or directory: '1000092795.jpg'

In [ ]:
# 모델 초기화
embed_size = 256
hidden_size = 512
vocab_size = 21652  # 단어 사전 크기
model = ImageCaptioningModel(embed_size, hidden_size, vocab_size)

# 추론 실행
output = model(image, caption)
